In [1]:
import numpy as np 
with open('pg2265.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text = text[15858:]
chars = set(text)
char2int = {ch:i for i, ch in enumerate(chars)}
int2char = dict(enumerate(chars))
text_ints = np.array([char2int[ch] for ch in text],dtype = np.int32)


In [2]:
def reshape_data(sequence, batch_size, num_steps):
    mini_batch_length = batch_size * num_steps
    num_batches = int(len(sequence) / mini_batch_length)
    if num_batches*mini_batch_length + 1 > len(sequence):
        num_batches = num_batches - 1

    x = sequence[0: num_batches*mini_batch_length]
    y = sequence[1: num_batches*mini_batch_length + 1]
    x_batch_splits = np.split(x, batch_size)
    y_batch_splits = np.split(y, batch_size)
    x = np.stack(x_batch_splits)
    y = np.stack(y_batch_splits)
    return x, y

In [3]:
def create_batch_generator(data_x, data_y, num_steps):
    batch_size, tot_batch_length = data_x.shape
    num_batches = int(tot_batch_length/num_steps)
    for b in range(num_batches):
        yield (data_x[:, b*num_steps:(b+1)*num_steps],
               data_y[:, b*num_steps:(b+1)*num_steps])

In [4]:
def get_top_char(probas, char_size, top_n=5):
    p = np.squeeze(probas)
    p[np.argsort(p)[:-top_n]] = 0.0
    p = p / np.sum(p)
    ch_id = np.random.choice(char_size, 1, p=p)[0]
    return ch_id

In [5]:
import tensorflow as tf
import os

class CharRNN(object):
    def __init__(self, num_classes, batch_size=64,
                 num_steps=100, lstm_size=128,
                 num_layers=1, learning_rate=0.001,
                 keep_prob=0.5, grad_clip=5,
                 sampling=False):
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.learning_rate = learning_rate
        self.keep_prob = keep_prob
        self.grad_clip = grad_clip
        
        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)

            self.build(sampling=sampling)

            self.saver = tf.train.Saver()

            self.init_op = tf.global_variables_initializer()
            
    def build(self, sampling):
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size = self.batch_size
            num_steps = self.num_steps

        tf_x = tf.placeholder(tf.int32,
                              shape=[batch_size, num_steps],
                              name='tf_x')
        tf_y = tf.placeholder(tf.int32,
                              shape=[batch_size, num_steps],
                              name='tf_y')
        tf_keepprob = tf.placeholder(tf.float32,
                              name='tf_keepprob')

        # One-hot encoding:
        x_onehot = tf.one_hot(tf_x, depth=self.num_classes)
        y_onehot = tf.one_hot(tf_y, depth=self.num_classes)

        ### Build the multi-layer RNN cells
        cells = tf.contrib.rnn.MultiRNNCell(
            [tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(self.lstm_size),
                output_keep_prob=tf_keepprob)
            for _ in range(self.num_layers)])
        
        ## Define the initial state
        self.initial_state = cells.zero_state(
                    batch_size, tf.float32)

        ## Run each sequence step through the RNN
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(
                    cells, x_onehot,
                    initial_state=self.initial_state)
        
        print('  << lstm_outputs  >>', lstm_outputs)

        seq_output_reshaped = tf.reshape(
                    lstm_outputs,
                    shape=[-1, self.lstm_size],
                    name='seq_output_reshaped')

        logits = tf.layers.dense(
                    inputs=seq_output_reshaped,
                    units=self.num_classes,
                    activation=None,
                    name='logits')

        proba = tf.nn.softmax(
                    logits,
                    name='probabilities')

        y_reshaped = tf.reshape(
                    y_onehot,
                    shape=[-1, self.num_classes],
                    name='y_reshaped')
        cost = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                        logits=logits,
                        labels=y_reshaped),
                    name='cost')

        # Gradient clipping to avoid "exploding gradients"
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(
                    tf.gradients(cost, tvars),
                    self.grad_clip)
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.apply_gradients(
                    zip(grads, tvars),
                    name='train_op')
        
    def train(self, train_x, train_y,
              num_epochs, ckpt_dir='./model/'):
        ## Create the checkpoint directory
        ## if it does not exists
        if not os.path.exists(ckpt_dir):
            os.mkdir(ckpt_dir)
            
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)

            n_batches = int(train_x.shape[1]/self.num_steps)
            iterations = n_batches * num_epochs
            for epoch in range(num_epochs):

                # Train network
                new_state = sess.run(self.initial_state)
                loss = 0
                ## Mini-batch generator:
                bgen = create_batch_generator(
                        train_x, train_y, self.num_steps)
                for b, (batch_x, batch_y) in enumerate(bgen, 1):
                    iteration = epoch*n_batches + b
                    
                    feed = {'tf_x:0': batch_x,
                            'tf_y:0': batch_y,
                            'tf_keepprob:0' : self.keep_prob,
                            self.initial_state : new_state}
                    batch_cost, _, new_state = sess.run(
                            ['cost:0', 'train_op',
                                self.final_state],
                            feed_dict=feed)
                    if iteration % 10 == 0:
                        print('Epoch %d/%d Iteration %d'
                              '| Training loss: %.4f' % (
                              epoch + 1, num_epochs,
                              iteration, batch_cost))

                ## Save the trained model
                self.saver.save(
                        sess, os.path.join(
                            ckpt_dir, 'language_modeling.ckpt'))

    def sample(self, output_length,
               ckpt_dir, starter_seq="The "):
        observed_seq = [ch for ch in starter_seq]
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(
                sess,
                tf.train.latest_checkpoint(ckpt_dir))
            ## 1: run the model using the starter sequence
            new_state = sess.run(self.initial_state)
            for ch in starter_seq:
                x = np.zeros((1, 1))
                x[0, 0] = char2int[ch]
                feed = {'tf_x:0': x,
                        'tf_keepprob:0': 1.0,
                        self.initial_state: new_state}
                proba, new_state = sess.run(
                        ['probabilities:0', self.final_state],
                        feed_dict=feed)

            ch_id = get_top_char(proba, len(chars))
            observed_seq.append(int2char[ch_id])
            
            ## 2: run the model using the updated observed_seq
            for i in range(output_length):
                x[0,0] = ch_id
                feed = {'tf_x:0': x,
                        'tf_keepprob:0': 1.0,
                        self.initial_state: new_state}
                proba, new_state = sess.run(
                        ['probabilities:0', self.final_state],
                        feed_dict=feed)

                ch_id = get_top_char(proba, len(chars))
                observed_seq.append(int2char[ch_id])

        return ''.join(observed_seq)
    
    


In [6]:
batch_size = 64
num_steps = 100
train_x, train_y = reshape_data(text_ints,
                             batch_size,
                               num_steps)

rnn = CharRNN(num_classes=len(chars), batch_size=batch_size)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
  << lstm_outputs  >> Tensor("rnn/transpose_1:0", shape=(64, 100, 128), dtype=float32)
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will a

In [7]:
rnn.train(train_x, train_y,
          num_epochs=100,
          ckpt_dir='./model-100/')

Epoch 1/100 Iteration 10| Training loss: 3.8463
Epoch 1/100 Iteration 20| Training loss: 3.3570
Epoch 2/100 Iteration 30| Training loss: 3.2839
Epoch 2/100 Iteration 40| Training loss: 3.2481
Epoch 2/100 Iteration 50| Training loss: 3.2278
Epoch 3/100 Iteration 60| Training loss: 3.2106
Epoch 3/100 Iteration 70| Training loss: 3.1942
Epoch 4/100 Iteration 80| Training loss: 3.1783
Epoch 4/100 Iteration 90| Training loss: 3.1478
Epoch 4/100 Iteration 100| Training loss: 3.1442
Epoch 5/100 Iteration 110| Training loss: 3.1291
Epoch 5/100 Iteration 120| Training loss: 3.0998
Epoch 6/100 Iteration 130| Training loss: 3.0602
Epoch 6/100 Iteration 140| Training loss: 3.0240
Epoch 6/100 Iteration 150| Training loss: 3.0061
Epoch 7/100 Iteration 160| Training loss: 2.9642
Epoch 7/100 Iteration 170| Training loss: 2.9210
Epoch 8/100 Iteration 180| Training loss: 2.8678
Epoch 8/100 Iteration 190| Training loss: 2.8349
Epoch 8/100 Iteration 200| Training loss: 2.7976
Epoch 9/100 Iteration 210| Tr

Epoch 66/100 Iteration 1650| Training loss: 1.9507
Epoch 67/100 Iteration 1660| Training loss: 1.9921
Epoch 67/100 Iteration 1670| Training loss: 1.9638
Epoch 68/100 Iteration 1680| Training loss: 1.9714
Epoch 68/100 Iteration 1690| Training loss: 1.9993
Epoch 68/100 Iteration 1700| Training loss: 1.9398
Epoch 69/100 Iteration 1710| Training loss: 1.9765
Epoch 69/100 Iteration 1720| Training loss: 1.9607
Epoch 70/100 Iteration 1730| Training loss: 1.9747
Epoch 70/100 Iteration 1740| Training loss: 1.9814
Epoch 70/100 Iteration 1750| Training loss: 1.9336
Epoch 71/100 Iteration 1760| Training loss: 1.9786
Epoch 71/100 Iteration 1770| Training loss: 1.9470
Epoch 72/100 Iteration 1780| Training loss: 1.9633
Epoch 72/100 Iteration 1790| Training loss: 1.9896
Epoch 72/100 Iteration 1800| Training loss: 1.9368
Epoch 73/100 Iteration 1810| Training loss: 1.9745
Epoch 73/100 Iteration 1820| Training loss: 1.9420
Epoch 74/100 Iteration 1830| Training loss: 1.9534
Epoch 74/100 Iteration 1840| Tr

In [12]:
del rnn

np.random.seed(123)
rnn = CharRNN(len(chars), sampling=True)
print("----------------------------------------------------")
print(rnn.sample(ckpt_dir='./model-100/', output_length=1000))

  << lstm_outputs  >> Tensor("rnn/transpose_1:0", shape=(1, 1, 128), dtype=float32)
----------------------------------------------------
INFO:tensorflow:Restoring parameters from ./model-100/language_modeling.ckpt
The wall steene,
When in the masse seate tis moning, bore as it thes,
That they so bathers of tile to seene and the ment

   Ham. Nor then, as is the word some the winge,
In tauth of hem the frander: and borthele
With all teere

   Hor. Thy the his sulle sis thim soule of the mast if
Wat it it thisers if my tore in inthing stoone, thingers
Toreete and the pases hare in this will to the math of,
And and thime ho steed of is, att thas as to heare:
I howe will the Singion and tomy alath and
Whit oure to touen thing andelles, and thouelesse,
And with that shand wording tell thing,
Thes word seale in orring hate ant moure,
The whall state, asting the mostes hing,
Thing to the this mane, how seefe, hit would,
And and thy thound, and whin sheadens,
In hare ow the the tonten

   Lair